In [74]:
import pandas as pd
import sqlite3
import matplotlib
%matplotlib inline

### Creating Helper Functions

In [75]:
def run_query(q):
    with sqlite3.connect('chinook.db') as conn:
        return pd.read_sql(q, conn)

def run_command(c):
    with sqlite3.connect('chinook.db') as conn:
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    q = '''
    SELECT * FROM sqlite_master 
    WHERE type IN ("table","view");
    '''
    df = run_query(q)
    return df[['name','type']]


#run_query('DROP VIEW view_name;')


In [76]:
show_tables()

,name,type
0,album,table
1,artist,table
2,customer,table
3,employee,table
4,genre,table
5,invoice,table
6,invoice_line,table
7,media_type,table
8,playlist,table
9,playlist_track,table


### Selecting Albums to Purchase
Write a query that returns each genre, with the number of tracks sold in absolute numbers and in percentages.

##### Notes:
1. Get total of each track sold: Sum Track ID from Invoice Line Table. ex:

Track ID | Quanity Sold 

In [82]:
run_command('DROP VIEW genre_quant;')

In [83]:
c = '''
CREATE VIEW genre_quant AS
SELECT il.track_id, il.quantity, t.genre_id, g.name, i.billing_country
FROM invoice_line il
INNER JOIN track t on t.track_id = il.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
INNER JOIN invoice i on i.invoice_id = il.invoice_id
'''
run_command(c)

In [ ]:
q = '''
WITH usa_tracks AS
(
SELECT name,
       quantity,
       billing_country
FROM genre_quant
WHERE billing_country="USA"
)
SELECT name genre,
       sum(quantity) total_sold,
       CAST(sum(quantity) AS float)/
       (SELECT sum(quantity)
       FROM genre_quant 
       WHERE billing_country="USA") AS percentage
FROM usa_tracks
GROUP BY genre
ORDER BY total_sold DESC
'''
df = run_query(q)
df.head(30)

In [68]:
# My answer is almost right, but I found global sales. 
# I need to filter to only include purchase from residents in the USA

In [106]:
q ='''
SELECT sum(quantity) FROM genre_quant WHERE billing_country="USA"
'''
run_query(q)

,sum(quantity)
0,1051


In [ ]:
WITH track_info AS
(
SELECT
t.name,
ar.name artist,
al.title album_name,
mt.name media_type,
g.name genre,
t.milliseconds length_milliseconds
FROM track t
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
INNER JOIN genre g ON g.genre_id = t.genre_id
INNER JOIN album al ON al.album_id = t.album_id
INNER JOIN artist ar ON ar.artist_id = al.artist_id
)

SELECT * FROM track_info
WHERE album_name = "Jagged Little Pill";